<a href="https://colab.research.google.com/github/PranoveAB/CourseraCertificateDownloader/blob/main/Coursera_Cert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install img2pdf

# New Section

In [ ]:
!pip install openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! cd /content/gdrive/


In [ ]:
import pandas as pd
import os
import requests
import re 
import img2pdf
import shutil
### Packages Required ###
# Pandas
# requests HTTP Library
# Openpyxl - To read directly from a .xlsx file   ### pip install openpyxl ### || Else convert the xlsx to .csv in Save As
# re for Regular Expressions
# Image to PDF  ### pip install img2pdf ###


df = pd.DataFrame(pd.read_excel("/content/gdrive/MyDrive/Original_Report.xlsx"))

#df = pd.read_csv('Original_Report.csv')

df.drop(df[df['Completed'] == 'No'].index, inplace = True)
df.dropna(subset = ["Course Certificate URL"], inplace=True)
df.drop(df.columns.difference(['Email','Course','Course Certificate URL']), 1, inplace=True)

# Some Courses have "":" in their names, cannot create files with ":", so reducing names of those files
df['Course'] = df['Course'].str.replace('[#,@,&,$,%,^,(,),/,_,-]',' ') # Removing special characters
#df['Course'] = df['Course'].str.split(':').str[0]

# Extract USN from Email ID
df['Usn'] = df['Email'].str.split('@').str[0]
# Extract Certificate Code from URL
df['CertCode'] = df['Course Certificate URL'].str.split('y/').str[1]

# Testing Purposes - take a small sample

#df = df[:20]
#print(df)


for i in range (len(df.index)):
    usn = df['Usn'].str.lower().tolist()
    certcode = df['CertCode'].tolist()
    courseName = df['Course'].tolist()

# To preview sample pre-processed data

#df.to_csv('new_csv.csv')

gdrivePath = '/content/gdrive/MyDrive/Coursera-Cert/'

# Add any new branches if I have missed out
branchesToCreate = ['cs','is','ec','ee','ai','me','te','mca','cv','faculty']
# Creating folders for different branches.
for i in range(len(branchesToCreate)):
    try:
        os.mkdir(gdrivePath+branchesToCreate[i])
    except OSError as error:
        print()

patternUsn = (re.compile(r"1by[0-9][0-9]*"))
patternBranch = (re.compile(r"cs|is|ec|ee|ai|me|te|mca|cv"))

# Downloading Each Certificate
def downloadfile(filepath):
    print("Downloading "+usn[i]+' certificate')
    with open(filepath,"wb+") as f:
        response.raw.decode_content = True
        #f.write((response.raw.read()))
        shutil.copyfileobj(response.raw, f)
      #f.write(img2pdf.convert(response))
      #f.write(response)

for i in range (len(df.index)):

    # Generic URL of Certificates. Each Certificate has its unique code, which was extracted from Course Certificate URL
    Urls = "https://s3.amazonaws.com/coursera_assets/meta_images/generated/CERTIFICATE_LANDING_PAGE/CERTIFICATE_LANDING_PAGE~"+certcode[i]+"/CERTIFICATE_LANDING_PAGE~"+certcode[i]+".jpeg"
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36'
    }
    response = requests.get(Urls, headers= headers, stream=True,verify=False)
    #response = requests.get(Urls,stream=True)


    if((bool(patternUsn.match(usn[i])))):

        # Downloading certificates to specific branch folders
        if((bool(patternBranch.match(usn[i][5:7])))):
            filepath = gdrivePath+usn[i][5:7]+"/"+usn[i]+"-"+str(courseName[i])+".jpeg"
            downloadfile(filepath)
        # Downloading certificates to MCA branch folder
        elif(usn[i][5:8]=="mca"):
            filepath = gdrivePath+"mca"+"/"+usn[i]+"-"+str(courseName[i])+".jpeg"
            downloadfile(filepath)
    # Downloading certificates to faculty branch folders. Added this condition cause it doesnt match with 1by**
    elif((bool(patternUsn.match(usn[i]))) == False):
            filepath = gdrivePath+"faculty"+"/"+usn[i]+"-"+str(courseName[i])+".jpeg"
            downloadfile(filepath)

FileNotFoundError: ignored